In [ ]:
from network import *
from transformer import *

data, tokenizer = loadTransformerData('data/', singular=True)
vocabSize = len(tokenizer.wordMap)
print("Vocabulary size:", vocabSize)

import tensorflow_datasets as tfds
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)
train_examples = examples['train']

for pt_examples, en_examples in train_examples.batch(20).take(1):
    print("Portuguese")
    for pt in pt_examples.numpy():
        print(pt.decode('utf-8'))
    print("English")
    for en in en_examples.numpy():
        print(en.decode('utf-8'))

In [ ]:
numLayers = 4
embeddingDepth = 128
feedDepth = 512
numHeads = 8

model = Transformer(numLayers=numLayers, vocabSize=vocabSize, embeddingDepth=embeddingDepth, 
                    feedDepth=feedDepth, numHeads=numHeads, tokenizer=tokenizer)
model.compile(loss=masked_loss, optimizer='adam', metrics=[masked_accuracy])

In [ ]:
model.fit(data, epochs=10)

In [ ]:
# model.save('extra/transformer.keras')

In [ ]:
# model = keras.models.load_model('extra/transformer.keras')

In [ ]:
def call(sentence, maxLength=256):
    tokenized = tokenizer.tokenize([sentence])
    sentence = tf.constant(tokenized)

    start = 1
    end = 2

    outputArray = [[start]]

    for i in range(maxLength):
        predictions = model.call([sentence, tf.constant(outputArray)])
        # print(predictions.shape)
        # print(tf.argmax(predictions[0][-1]))

        predicted_id = tf.argmax(predictions[0][-1]).numpy()
        outputArray[0].append(predicted_id)

        if predicted_id == end:
            break

    return outputArray

In [ ]:
from transformer import Translator
# translator = Translator(tokenizer, model)
output = call(input("Chat with Chicken: "))